#**Part 1 - Data gathering and preprocessing**


**Libraries**

In [0]:
import numpy as np #Linear_Algebra
import matplotlib.pyplot as plt
import pandas as pd #Data_Processing
import pandas_datareader as pdr
from scipy import stats
%matplotlib inline

In [0]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [0]:
pip install -q yfinance --upgrade

In [0]:
#Import Yahoo Finance
import yfinance as yf

In [0]:
yf.pdr_override()

In [0]:
#CISCO data
SELECTED_STOCK = 'CSCO'
start = '2010-12-17'
end = '2019-12-17'

In [0]:
#Download NVIDIA stock price data for the past 10 yrs to date
stock_data = pdr.get_data_yahoo(SELECTED_STOCK, start, end)

In [8]:
stock_data.head(10)

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2010-12-17,19.780001,19.500000,19.700001,19.549999,115525000.0,15.245194
2010-12-20,19.719999,19.520000,19.580000,19.620001,49113500.0,15.299781
2010-12-21,19.770000,19.500000,19.639999,19.530001,62386300.0,15.229600
2010-12-22,19.570000,19.510000,19.540001,19.559999,42787100.0,15.252996
2010-12-23,19.760000,19.500000,19.540001,19.690001,41599200.0,15.354370
2010-12-27,20.400000,19.809999,19.840000,20.160000,62054900.0,15.720881
2010-12-28,20.510000,20.280001,20.370001,20.350000,57275400.0,15.869036
2010-12-29,20.510000,20.250000,20.490000,20.250000,32977000.0,15.791062
2010-12-30,20.350000,20.209999,20.320000,20.230000,25576700.0,15.775463


**Feature Engineering**

In [9]:
#Getting the Open price
stock_data_open = stock_data.Open.values
reshaped_stock_data_open = np.reshape(stock_data_open, (-1, 1))
reshaped_stock_data_open

array([[19.70000076],
       [19.57999992],
       [19.63999939],
       ...,
       [44.54999924],
       [45.90999985],
       [45.79999924]])

In [10]:
#validity check
np.mean(reshaped_stock_data_open)==np.mean(stock_data_open)

True

**Feature Scaling**

In [0]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0,1))
scaled_data = sc.fit_transform(reshaped_stock_data_open)

In [0]:
def timestamp(n_period, scaled_data):
    x_train = [] 
    y_train = [] #1 output to predict

    for i in range(n_period,len(scaled_data)):
        x_train.append(scaled_data[i-n_period:i,0])
        y_train.append(scaled_data[i,0])
    x_train, y_train = np.array(x_train), np.array(y_train)

    #reshaping
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

    return x_train, y_train

In [0]:
x_train, y_train = timestamp(60, scaled_data)

#**Part 2 - Building the RNN**

In [0]:
import warnings
warnings.simplefilter("ignore")

In [0]:
#Importing the keras libraries and packages
from tensorflow.python.keras.layers import Dense, LSTM, Dropout
from tensorflow.python.keras import Sequential

In [0]:
regressor = Sequential()

#Adding the first LSTM Layer and some Dropout regularisation
regressor.add(LSTM(units=50, return_sequences=True, input_shape = (x_train.shape[1], 1)))
regressor.add(Dropout(rate = 0.2))

In [21]:
x_train.shape[1]

60

In [0]:
#Adding the second LSTM Layer and some Dropout regularisation
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(rate = 0.2))

In [0]:
#Adding the third LSTM Layer and some Dropout regularisation
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(rate = 0.2))

In [0]:
#Adding the fourth LSTM Layer and some Dropout regularisation
regressor.add(LSTM(units=50))
regressor.add(Dropout(rate = 0.2))

In [0]:
#Adding the output layer
regressor.add(Dense(units=1))

In [0]:
#compiling the RNN
regressor.compile(optimizer='adam', loss='mean_squared_error')

In [27]:
#fitting the RNN to the training set
regressor.fit(x_train, y_train, epochs=50, batch_size = 32)

Train on 2204 samples
Epoch 1/50
2204/2204 [==============================] - 28s 13ms/sample - loss: 0.0181
Epoch 2/50
2204/2204 [==============================] - 18s 8ms/sample - loss: 0.0029
Epoch 3/50
2204/2204 [==============================] - 18s 8ms/sample - loss: 0.0032
Epoch 4/50
2204/2204 [==============================] - 18s 8ms/sample - loss: 0.0026
Epoch 5/50
2204/2204 [==============================] - 18s 8ms/sample - loss: 0.0025
Epoch 6/50
2204/2204 [==============================] - 18s 8ms/sample - loss: 0.0022
Epoch 7/50
2204/2204 [==============================] - 18s 8ms/sample - loss: 0.0024
Epoch 8/50
2204/2204 [==============================] - 18s 8ms/sample - loss: 0.0021
Epoch 9/50
2204/2204 [==============================] - 17s 8ms/sample - loss: 0.0022
Epoch 10/50
2204/2204 [==============================] - 18s 8ms/sample - loss: 0.0021
Epoch 11/50
2204/2204 [==============================] - 17s 8ms/sample - loss: 0.0024
Epoch 12/50
2204/2204 [======

**Save the model**

In [0]:
regressor = regressor.save("regressor.h5")

**Load the model**

In [0]:
from tensorflow.python.keras.models import load_model

regressor = load_model("regressor.h5")

#**Part 3 - Making the predictions and visualising the results**